In [1]:
using Pkg; Pkg.activate("/home/dhairyagandhi96/modelzoo/model-zoo/script/../other/fizzbuzz"); Pkg.instantiate(); Pkg.add(PackageSpec(name="CUDAdrv", rev = "master")); Pkg.add(PackageSpec(name="CUDAnative", rev = "master")); Pkg.add(PackageSpec(name="CuArrays", rev = "master")); Pkg.add(PackageSpec(name="Flux", rev = "master"))

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/JuliaGPU/CUDAdrv.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/other/fizzbuzz/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/other/fizzbuzz/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CUDAnative.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/other/fizzbuzz/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/other/fizzbuzz/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CuArrays.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/other/fizzbuzz/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/other/fizzbuzz/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/FluxML/Flux.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/oth

Inspired by "Fizz Buzz in Tensorflow" blog by Joel Grus
http://joelgrus.com/2016/05/23/fizz-buzz-in-tensorflow/

In [2]:
using Flux: Chain, Dense, params, crossentropy, onehotbatch,
            ADAM, train!, softmax
using Test

Data preparation

In [3]:
function fizzbuzz(x::Int)
    is_divisible_by_three = x % 3 == 0
    is_divisible_by_five = x % 5 == 0

    if is_divisible_by_three & is_divisible_by_five
        return "fizzbuzz"
    elseif is_divisible_by_three
        return "fizz"
    elseif is_divisible_by_five
        return "buzz"
    else
        return "else"
    end
end

const LABELS = ["fizz", "buzz", "fizzbuzz", "else"];

@test fizzbuzz.([3, 5, 15, 98]) == LABELS

raw_x = 1:100;
raw_y = fizzbuzz.(raw_x);

Feature engineering

In [4]:
features(x) = float.([x % 3, x % 5, x % 15])
features(x::AbstractArray) = hcat(features.(x)...)

X = features(raw_x);
y = onehotbatch(raw_y, LABELS);

Model

In [5]:
m = Chain(Dense(3, 10), Dense(10, 4), softmax)
loss(x, y) = crossentropy(m(X), y)
opt = ADAM(params(m))

┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = top-level scope at none:0
└ @ Core none:0


#13 (generic function with 1 method)

Helpers

In [6]:
deepbuzz(x) = (a = argmax(m(features(x))); a == 4 ? x : LABELS[a])

function monitor(e)
    print("epoch $(lpad(e, 4)): loss = $(round(loss(X,y).data; digits=4)) | ")
    @show deepbuzz.([3, 5, 15, 98])
end

monitor (generic function with 1 method)

Training

In [7]:
for e in 0:1000
    train!(loss, [(X, y)], opt)
    if e % 50 == 0; monitor(e) end
end

┌ Warning: train!(loss, data, opt) is deprecated; use train!(loss, params, data, opt) instead
│   caller = top-level scope at string:2 [inlined]
└ @ Core ./string:2
epoch    0: loss = 1.68 | deepbuzz.([3, 5, 15, 98]) = Any[3, 5, "fizzbuzz", 98]
epoch   50: loss = 0.7885 | deepbuzz.([3, 5, 15, 98]) = Any[3, "buzz", "fizzbuzz", 98]
epoch  100: loss = 0.6143 | deepbuzz.([3, 5, 15, 98]) = Any[3, "buzz", "fizzbuzz", 98]
epoch  150: loss = 0.5202 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", "buzz", "fizzbuzz", 98]
epoch  200: loss = 0.4486 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", "buzz", "fizzbuzz", 98]
epoch  250: loss = 0.3885 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", "buzz", "fizzbuzz", 98]
epoch  300: loss = 0.3366 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", "buzz", "fizzbuzz", 98]
epoch  350: loss = 0.2916 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", "buzz", "fizzbuzz", 98]
epoch  400: loss = 0.2527 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", "buzz", "fizzbuzz", 98]
epoch  450: loss = 0.2192 | d